In [1]:
import numpy as np

In [5]:
def softmax(x):
    ex = np.exp(x-np.max(x))
    return ex / ex.sum(axis = 0)

In [9]:
def rnn_cell_forward(xt, a_prev, parameters):
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    y_pred = softmax(np.dot(Wya, a_next) + by)
    
#     cache = (a_next, a_prev, xt, parameters)
    
    return a_next, y_pred

In [10]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 50):
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0
    
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size, 1))
        x[t][X[t]] = 1
        
        a[t], y_hat[t] = rnn_cell_forward(x[t], a[t-1], parameters)
        
        loss = -np.log(y_hat[t][Y[t],0])
    
    cache = (y_hat, a, x)
    return loss, cache

In [ ]:
def rnn_cell_backward(da_next, cache):
    (a_next, a_prev, xt, parameters) = cache
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    